In [1]:
import nbformat
from nbformat import v4 as nbf

### Tablas a auditar (editar)
> Por cada tabla agregar un elemento a la lista, cada elemento de la lista es un diccionario.<br>Los elementos de dicho diccionarios son:
>- **ram**: nombre de la tabla en raw
>- **master**: nombre de la tabla en master
>- **rawRules**: numero de las reglas aplicadas en raw 
>- **masterRules**: numero de las reglas aplicadas en master

> Las reglas generadas estarán contenidas en otro notebook llamado **"auditoria_notebook.ipynb"**. Cada ejecución lo reescribirá.

Elaboró:<br> **Daniel García
<br>daniel.garcia@tuta.io**

In [2]:
tables = [
    {"raw": "t_crm_regresos_operadores_smc_c", "master": "t_ft1l_regresos_operadores_smc",
     "rawRules": ["2.1", "2.4", "2.2"], "masterRules": ["1.1", "2.3", "3.1"]},
    {"raw": "t_crm_regresos_operadores_ato_c", "master": "t_ft1l_regresos_operadores_ato",
     "rawRules": ["2.1", "2.4", "2.2"], "masterRules": ["1.1", "2.3", "3.1"]},
    {"raw": "t_crm_regresos_operadores_muz_c", "master": "t_ft1l_regresos_operadores_muz",
     "rawRules": ["2.1", "2.4", "2.2"], "masterRules": ["1.1", "2.3", "3.1"]},
    {"raw": "t_crm_regresos_operadores_vcp_c", "master": "t_ft1l_regresos_operadores_vcp",
     "rawRules": ["2.1", "2.4", "2.2"], "masterRules": ["1.1", "2.3", "3.1"]}
]

### Catalogo de reglas de Hammurabi

In [3]:
rulesCatalog = {
    "1.1": ["Availability","com.rules.availability.DateRangeFileRule"],
    "1.2": ["Availability","com.rules.availability.DateValidationRule"],
    "2.1": ["Completeness","com.rules.completeness.CompletenessRule"],
    "2.2": ["Completeness","com.rules.completeness.BasicPerimeterCompletenessRule"],
    "2.3": ["Completeness","com.rules.completeness.ConditionalPerimeterCompletenessRule"],
    "2.4": ["Completeness","com.rules.completeness.MetadataCompletenessRule"],
    "3.1": ["Validity","com.rules.validity.NotNullValidationRule"],
    "3.2": ["Validity","com.rules.validity.FormatValidationRule"],
    "3.3": ["Validity","com.rules.validity.StaticForbiddenValuesRule"],
    "3.4": ["Validity","com.rules.validity.NumericRangeRule"],
    "3.5": ["Validity","com.rules.validity.StaticCatalogRule"],
    "4.2": ["Consistence","com.datio.consistence.DuplicateRule"],
    "4.3": ["Consistence","com.datio.consistence.ValueConciliationRule"],
    "5.2": ["Integrity","com.rules.integrity.ValueComparisonRule"],
    "6.1": ["Accuracy","com.rules.accuracy.StaticGatheringRule"],
    "6.2": ["Accuracy","com.rules.accuracy.ValueSensitivityThresholdRule"],
    "6.4": ["Accuracy","com.rules.accuracy.ValueVariationRule"],
    "6.9": ["Accuracy","com.rules.accuracy.VolumetricTrendRule"]
}

### Lectura de las estadisticas y columnas a seleccionar

In [4]:
initHammurabi = f"""val hamm = spark.read.option("mergeSchema", "true").parquet("/data/master/dq/haas/t_kqpd_stats/*")"""
columns = f'''\t"gf_cutoff_date",
    "g_quality_rule_principle_type",
    "g_quality_rule_type",
    "gf_quality_rule_compliance_per",         
    "gf_qr_cplc_numerator_number",   
    "gf_qr_cplc_denominator_number",  
    "gf_qr_tg_object_physical_name",
    "gf_qr_aux_attribute_desc",   
    "gf_qr_aux_attribute1_desc"'''

### Consulta a ejecutar en la tabla

In [5]:
query = '''hamm.filter(
    (($"g_quality_rule_principle_type"  === "{}")  &&
    ($"g_quality_rule_type"    === "{}")) &&
    ($"gf_qr_tg_object_physical_name" === "{}")
)
.select(
{}
).show(false)'''

### Función para crear las reglas de consulta

In [6]:
def createQueries(table, rules):
    for rule in rules:
        values = rule.split(".")
        cells.extend([
            nbf.new_markdown_cell("### Regla "+rule+": "+ rulesCatalog[rule][1].rsplit('.',1)[1]),
            nbf.new_code_cell(query.format(values[0], values[1], table, columns))
        ])

### Escritura del encabezado e imports

In [7]:
nb = nbf.new_notebook()
cells = []
cells.extend([
    nbf.new_markdown_cell("# Cifras de Auditoria"),
    nbf.new_code_cell(initHammurabi)
])

### Escritura de reglas en el notebook

In [8]:
for table in tables:
    cells.extend([
        nbf.new_markdown_cell("***"),
        nbf.new_markdown_cell("# Tablas:\n* " + table["raw"] +"\n* " + table["master"])
    ])
    cells.append(nbf.new_markdown_cell("## Raw: "+table["raw"]))
    createQueries(table["raw"], table["rawRules"])
    cells.append(nbf.new_markdown_cell("## Master: "+table["master"]))
    createQueries(table["master"], table["masterRules"])

### Creación del nuevo notebook

In [9]:
nb.cells.extend(cells)
with open('auditoria_notebook.ipynb', 'w') as f:
    nbformat.write(nb, f)